In [1]:
import osmnx as ox
import networkx as nx
import numpy as np
import pandas as pd
import geopandas as gpd
import random
from shapely.geometry import Point
from pathlib import Path

In [7]:
def cargar_geojson(nombre_archivo, directorio_base='../../../data/delivetter'):
    ruta = Path(directorio_base) / nombre_archivo
    return gpd.read_file(ruta)

In [ ]:
barrios = cargar_geojson('barris.geojson')
catastro = cargar_geojson('catastro.geojson')
puntos_carga = cargar_geojson('puntosCID.geojson')

In [ ]:
comercios = [8, 10, 12]
capacidades = [6, 8, 10]
vs_vehiculo = [15, 20]
vs_peaton = [4, 5]
barrios = ["Benimaclet", "El botanic", "Sant Francesc"]

### Generate minable view of model 1

In [22]:
def run_simulation_model1(
    barrio: str,
    N_COMERCIOS=10,
    N_CID=3,
    PAQUETES_MIN=1,
    PAQUETES_MAX=5,
    CAPACIDAD_MAXIMA=10,
    VELOCIDAD_VEHICULO=20,  # km/h
    VELOCIDAD_PEATON=5,     # km/h
):
    shp_neighbourhood = barrios[barrios["nombre"].str.upper().str.contains(barrio.strip().upper())]
    shp_zone = shp_neighbourhood["geometry"].values[0]
    shp_loading_points = puntos_carga[puntos_carga.intersects(shp_zone)]
    shp_bajos = catastro[catastro.intersects(shp_zone)]

    G = ox.graph_from_polygon(shp_zone, network_type="all_public", simplify=False)
    allowed_vehicle = ['motorway', 'residential', 'secondary', 'living_street', 'primary_link', 'primary', 'tertiary', 'trunk', 'service']
    allowed_pedestrian = ["pedestrian", "footway", "path", "track", "bridleway", "living_street", "residential"]

    G_vehicle = G.copy()
    G_pedestrian = G.copy()

    G_vehicle.remove_edges_from([(u,v,k) for u,v,k,d in G_vehicle.edges(keys=True, data=True) if d.get('highway') not in allowed_vehicle])
    G_pedestrian.remove_edges_from([(u,v,k) for u,v,k,d in G_pedestrian.edges(keys=True, data=True) if d.get('highway') not in allowed_pedestrian])

    from shapely.geometry import Point, Polygon

    # Elegimos comercios
    comercios = shp_bajos[shp_bajos['Comercio'] == True].copy()
    comercios_sample = comercios.sample(n=N_COMERCIOS, random_state=42).copy()
    comercios_sample['paquetes'] = np.random.randint(PAQUETES_MIN, PAQUETES_MAX + 1, size=N_COMERCIOS)

    # Elegimos puntos de carga
    puntos_carga_sample = shp_loading_points.sample(n=N_CID, random_state=42).copy()

    # Añadir nodos al grafo general
    G_super: nx.MultiDiGraph = nx.compose(G_vehicle, G_pedestrian)
    for u, _, _, data in G_vehicle.edges(keys=True, data=True):
        data['mode'] = 'drive'
    for u, _, _, data in G_pedestrian.edges(keys=True, data=True):
        data['mode'] = 'walk'

    # Añadir nodos especiales
    def add_special_node(G: nx.MultiDiGraph, geom: Polygon, tipo: str, id_: int):
        nodo_id = f"{tipo}_{id_}"
        G.add_node(nodo_id, x=geom.centroid.x, y=geom.centroid.y, tipo=tipo)
        grafo_conexion = G_vehicle if tipo == "carga" else G_pedestrian
        nearest = ox.nearest_nodes(grafo_conexion, geom.centroid.x, geom.centroid.y)
        data = grafo_conexion.nodes[nearest]
        distance = Point(geom.centroid.x, geom.centroid.y).distance(Point(data['x'], data['y'])) * 111000
        G.add_edge(nodo_id, nearest, mode='connection', length=distance)
        G.add_edge(nearest, nodo_id, mode='connection', length=distance)
        return nodo_id

    comercios_sample['nodo'] = [add_special_node(G_super, row.geometry, "comercio", i) for i, row in comercios_sample.iterrows()]
    puntos_carga_sample['nodo'] = [add_special_node(G_super, row.geometry, "carga", i) for i, row in puntos_carga_sample.iterrows()]

    # Asignar punto de carga más cercano a cada comercio
    def distancia_min(nodo_comercio):
        distancias = []
        for nodo_carga in puntos_carga_sample['nodo']:
            try:
                path = nx.shortest_path(G_super, nodo_carga, nodo_comercio, weight='length')
                dist = sum(G_super.edges[path[i], path[i+1], 0]['length'] for i in range(len(path)-1))
                distancias.append((dist, nodo_carga))
            except:
                continue
        return min(distancias, key=lambda x: x[0])[1] if distancias else None

    comercios_sample['carga_nodo'] = comercios_sample['nodo'].apply(distancia_min)

    # Simulación de entregas
    tiempo_drive = 0
    tiempo_walk = 0

    for carga_id, grupo in comercios_sample.groupby('carga_nodo'):
        try:
            path = nx.shortest_path(G_super, carga_id, grupo.iloc[0]['nodo'], weight='length')
            distancia = sum(G_super.edges[path[i], path[i+1], 0]['length'] for i in range(len(path)-1))
            tiempo_drive += distancia / (VELOCIDAD_VEHICULO * 1000 / 60)
        except:
            continue
        for _, row in grupo.iterrows():
            try:
                path = nx.shortest_path(G_super, carga_id, row['nodo'], weight='length')
                distancia = sum(G_super.edges[path[i], path[i+1], 0]['length'] for i in range(len(path)-1))
                tiempo_walk += 2 * distancia / (VELOCIDAD_PEATON * 1000 / 60)  # ida y vuelta
            except:
                continue

    tiempo_total = tiempo_drive + tiempo_walk

    return {
        "model_id": 1,
        "vehicle": "van",
        "neighborhood": barrio,
        "num_stores": N_COMERCIOS,
        "num_cid": N_CID,
        "max_capacity": CAPACIDAD_MAXIMA,
        "vehicle_speed": VELOCIDAD_VEHICULO,
        "pedestrian_speed": VELOCIDAD_PEATON,
        "total_time": round(tiempo_total, 2),
        "drive_time": round(tiempo_drive, 2),
        "walk_time": round(tiempo_walk, 2),
        "package_efficiency": round(tiempo_total / comercios_sample['paquetes'].sum(), 2)
        }


# Ejecutar batería de simulaciones y guardar resultados

def generar_minable_view_model1(save=False):
    resultados = []

    for n_comercios in comercios:
        for capacidad in capacidades:
            for v_vehiculo in vs_vehiculo:
                for v_peaton in vs_peaton:
                    for barrio in barrios:
                        try:
                            resultado = run_simulation_model1(
                                barrio=barrio,
                                N_COMERCIOS=n_comercios,
                                CAPACIDAD_MAXIMA=capacidad,
                                VELOCIDAD_VEHICULO=v_vehiculo,
                                VELOCIDAD_PEATON=v_peaton
                            )
                            resultados.append(resultado)
                        except Exception as e:
                            raise e

    df = pd.DataFrame(resultados)
    
    if save:
        df.to_csv("minable_view_model1.csv", index=False)
        print("✅ Minable view guardada como minable_view_model1.csv")
    return df

df_model_1 = generar_minable_view_model1()

KeyboardInterrupt: 

### Generate minable view of model 3

In [ ]:
def run_simulation_model3(
    barrio: str,
    N_COMERCIOS=10,
    PAQUETES_MIN=1,
    PAQUETES_MAX=5,
    VELOCIDAD_ROBOT=5,
    CAPACIDAD_ROBOT=8,
):
    shp_neighbourhood = barrios[
        barrios["nombre"].str.upper().str.contains(barrio.strip().upper())
    ]
    shp_zone = shp_neighbourhood["geometry"].values[0]
    shp_bajos = catastro[catastro.intersects(shp_zone)]

    G = ox.graph_from_polygon(shp_zone, network_type="all_public", simplify=False)
    G = ox.project_graph(G)

    # Añadir nodos especiales
    def add_special_node(G, geom, tipo, id_):
        from shapely.geometry import Point

        nearest = ox.nearest_nodes(G, geom.centroid.x, geom.centroid.y)
        nodo_id = nearest if tipo == "comercio" else f"{tipo}_{id_}"
        if nodo_id not in G.nodes:
            G.add_node(nodo_id, x=geom.centroid.x, y=geom.centroid.y, tipo=tipo)
            data = G.nodes[nearest]
            distance = Point(geom.centroid.x, geom.centroid.y).distance(
                Point(data["x"], data["y"])
            )
            G.add_edge(nodo_id, nearest, mode="connection", length=distance)
            G.add_edge(nearest, nodo_id, mode="connection", length=distance)
        return nodo_id

    bajos_entrega = shp_bajos[shp_bajos["Comercio"] == True].copy()
    bajos_sample = bajos_entrega.sample(n=N_COMERCIOS, random_state=42).copy()
    bajos_sample["paquetes"] = np.random.randint(
        PAQUETES_MIN, PAQUETES_MAX + 1, size=N_COMERCIOS
    )
    bajos_sample["nodo"] = [
        add_special_node(G, row.geometry, "comercio", i)
        for i, row in bajos_sample.iterrows()
    ]

    centroide_barrio = shp_zone.centroid
    almacenes = shp_bajos[shp_bajos["Almacen"] == True].copy()
    almacenes["dist"] = almacenes.geometry.centroid.distance(centroide_barrio)
    hub_geom = almacenes.sort_values("dist").iloc[0].geometry
    hub_nodo = add_special_node(G, hub_geom, "almacen", 0)

    ruta_total = []
    tiempo_total = 0
    pos_actual = hub_nodo
    pendientes = bajos_sample.copy()

    while not pendientes.empty:
        carga_actual = 0
        entregas_ruta = []

        for idx, row in pendientes.iterrows():
            nodo_comercio = row["nodo"]
            paquetes = row["paquetes"]

            if carga_actual + paquetes <= CAPACIDAD_ROBOT:
                if nx.has_path(G, pos_actual, nodo_comercio):
                    entregas_ruta.append(idx)
                    carga_actual += paquetes

        if not entregas_ruta:
            break

        for idx in entregas_ruta:
            nodo_comercio = pendientes.loc[idx, "nodo"]
            try:
                camino = nx.shortest_path(G, pos_actual, nodo_comercio, weight="length")
                distancia = sum(
                    G.edges[camino[i], camino[i + 1], 0]["length"]
                    for i in range(len(camino) - 1)
                )
                tiempo = distancia / (VELOCIDAD_ROBOT * 1000 / 60)
                tiempo_total += tiempo
                ruta_total += camino[1:]
                pos_actual = nodo_comercio
            except Exception as e:
                continue

        pendientes = pendientes.drop(index=entregas_ruta)

    return {
        "model_id": 3,
        "vehicle": "robot",
        "neighborhood": barrio,
        "num_stores": N_COMERCIOS,
        "max_capacity": CAPACIDAD_ROBOT,
        "vehicle_speed": VELOCIDAD_ROBOT,
        "total_time": round(tiempo_total, 2),
        "drive_time": round(tiempo_total, 2),
        "package_efficiency": round(tiempo_total / bajos_sample["paquetes"].sum(), 2),
    }


# Ejecutar batería de simulaciones y guardar resultados


def generar_minable_view_model3(save=False):
    import pandas as pd

    resultados = []

    for n_comercios in comercios:
        for capacidad in capacidades:
            for velocidad in vs_vehiculo:
                for barrio in barrios:
                    try:
                        resultado = run_simulation_model3(
                            barrio=barrio,
                            N_COMERCIOS=n_comercios,
                            CAPACIDAD_ROBOT=capacidad,
                            VELOCIDAD_ROBOT=velocidad,
                        )
                        resultados.append(resultado)
                    except Exception as e:
                        print(f"❌ Error en simulación: {e}")

    df = pd.DataFrame(resultados)

    if save:
        df.to_csv("minable_view_model3.csv", index=False)
        print("✅ Minable view guardada como minable_view_model3.csv")
    return df


df_model_3 = generar_minable_view_model3()

### Unified view of model 1 and model 2

In [ ]:
import pandas as pd

# Unificar columnas ausentes (rellena con NaN donde falte)
df_unificado = pd.concat([df_model_1, df_model_3], ignore_index=True, sort=False)

# Guardar como nuevo archivo
df_unificado.to_csv("minable_view_combined.csv", index=False)
print("✅ Archivo combinado guardado como minable_view_combined.csv")


✅ Archivo combinado guardado como minable_view_combined.csv
